In [1]:
import pickle
import plotly.graph_objects as go
from copy import copy, deepcopy
from pprint import pprint
from Geometry3D import Point, distance, HalfLine, Plane
import Geometry3D
import gcline
from importlib import reload

#import numpy as np
#suppress scientific notation
#np.set_printoptions(suppress=True, precision=8)
#import parsers.cura4
#from parsers import cura4

import threader, geometry_helpers, gcode, gcline, tlayer, util, cura4layer, lablogging, rich_output_handler, logger, plot_styles
from geometry_helpers import GPoint
from threader import Threader
from tlayer import TLayer
from util import find, GCodeException
from danutil import dupcount
import plot_helpers, plotting, plot_styles
from plotting import plot_steps
import geometry.utils

#Attempt to reload modules automatically
%load_ext autoreload
%autoreload 2

reload threader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#Fusion: smaller complex shape v1
gcode_file = '/Users/dan/r/thread_printer/stl/test2/Smaller complex shape - cura.gcode'
cura_project = '/Users/dan/r/thread_printer/stl/test2/Smaller complex shape - cura.3mf'
# From cura4.parse_3mf(cura_project) so I don't have to run that every time
thread_transform = np.array([39.26039839, 46.27034378, 23.87050056])
#See thread_from_fusion.py to get thread path
tpath = [
		([33.7300, 17.1077, 11.4653], [56.0477, 43.4545, 11.4653]),
		([56.0477, 43.4545, 11.4653], [53.2410, 59.2656, 11.4653]),
		([53.2410, 59.2656, 11.4653], [26.9901, 69.3288, 42.0560]),
		([26.9901, 69.3288, 42.0560], [26.9901, 69.3288, 47.7410]),
]

In [2]:
gcode_file = '/Users/dan/r/thread_printer/stl/test3/Router test-cura5.gcode'
cura_project = '/Users/dan/r/thread_printer/stl/test3/Router test-cura5.3mf'
#thread_transform = np.array([27.43560219, 27.43383789,  5.0])
tpath = [
	([18.2689, 14.9338, 3.3333], [18.2689, 36.6005, 3.3333]),
	([18.2689, 36.6005, 3.3333], [36.6023, 36.6005, 3.3333]),
	([36.6023, 36.6005, 3.3333], [36.6023, 18.2672, 3.3333]),
	([36.6023, 18.2672, 3.3333], [21.6023, 18.2672, 3.3333]),
	([21.6023, 18.2672, 3.3333], [21.6023, 21.6005, 6.6667]),
	([21.6023, 21.6005, 6.6667], [21.6023, 36.6005, 6.6667]),
	([21.6023, 36.6005, 6.6667], [33.2689, 36.6005, 6.6667]),
	([33.2689, 36.6005, 6.6667], [33.2689, 21.6005, 6.6667])
]

In [ ]:
thread_transform = cura4.parse_3mf(cura_project)
thread_transform

In [6]:
#tpath = np.array(tpath)

#Do it this way so I can add transform if needed
#tpath = np.insert(tpath, 0, [[0,0,0], tpath[0,0]], axis=0)

#Temporary fix for TODO in cura4.parse3mf
#thread_transform[2] = 0

g = gcode.GcodeFile(gcode_file, layer_class=TLayer)
t = Threader(g)

#start_layer = next(layer for layer in g.layers if (layer.z + layer.layer_height) >= tpath[0][1][2])
#start_layer = g.layers[56]

thread_geom = list([geometry_helpers.GSegment(GPoint(*s), GPoint(*e)) for s,e in tpath])

In [7]:
t.route_model(thread_geom, end_layer=17)

Accordion()

In [9]:
l = 32
#reload(plot_styles)
#reload(plotting)
#plotting.styles = plot_styles.styles
plotting.plot_steps(t.layer_steps[l], prev_layer=g.layers[l-1], prev_layer_only_outline=False)

IndexError: list index out of range

In [8]:
step = t.layer_steps[16].steps[1]
step.gcode()

[G0 X15.536 Y15.534 ; [] ---- Fake move from 4502,
 [4503] G1 X15.536 Y24.334 E420.28914 ; [4503],
 G0 X15.136 Y15.134 ; [] ---- Skipped 4504–4508; fake move from 4508,
 [4509] G1 X15.136 Y24.734 E421.53972 ; [4509],
 [4514] G0 F9000 X15.806 Y15.804 ; [4514],
 [4515] G0 X15.836 Y17.465 ; [4515],
 [4516] G0 X15.615 Y17.465 ; [4516],
 [4517] G1 F3000 X17.467 Y15.613 E422.35253 ; [4517],
 [4520] G0 F9000 X24.255 Y22.402 ; [4520],
 [4521] G1 F3000 X22.403 Y24.253 E422.73957 ; [4521],
 [4530] G0 F9000 X24.255 Y20.139 ; [4530],
 [4531] G1 F3000 X20.141 Y24.253 E423.62008 ; [4531],
 [4538] G0 F9000 X22.403 Y15.614 ; [4538],
 [4539] G1 F3000 X24.255 Y17.465 E424.20061 ; [4539],
 [4542] G0 F9000 X17.466 Y24.253 ; [4542],
 [4543] G1 F3000 X15.615 Y22.402 E424.58761 ; [4543],
 [4548] G0 F9000 X23.736 Y16.016 ; [4548],
 [4549] G1 F1500 E418.49396 ; [4549],
 [4550] ; [4550] MESH:Router test.3mf(4),
 [4551] G0 F9000 X24.516 Y24.064 ; [4551],
 [4552] G0 X25.601 Y24.064 ; [4552],
 [4553] G0 X29.271 Y2